In [1]:
import numpy as np
import pandas as pd
import os
import glob

In [2]:
###### Enable if debugging, to reflect changes in .py files without restarting kernel #####
# %reload_ext autoreload
# %autoreload 2

from utils.data_utils import read,read_parts,read_dict,read_json_dump,read_dict_dump,load_config, load_hyperparams
from utils.sample_threads import get_threads_keywords,filter_threads,print_thread,read_thread_file
from utils.thread_utils import get_thread_file_pattern, eval_filter_threads

from seqint.run_seqint import run_seqint
from hint.run_hint import run_hint
# from seqint.eval_hac_5w1h import get_threads,eval_ev_threads

from dpp.dpp_preprocess import compute_passage_cosine,create_graphs,convert_to_mat
from dpp.run_dpp import run_ksdpp,eval_sdpp_threads
# from dpp.dpp_eval import sddp_read_threads,eval_ksdpp

2023-02-17 19:48:04.032969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 19:48:04.669213: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-17 19:48:04.669286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-17 19:48:04.669293: W tensorfl

In [27]:
config_override=dict(transformer_dir="/nfs/jup/sensitivity_classifier/threading/e2e_data/transformer_models",
source_dir="/nfs/jup/sensitivity_classifier/threading/e2e_2023/92Folder",
target_dir="/nfs/jup/sensitivity_classifier/threading/e2e_data/")

In [ ]:
config_override={}

config=load_config(overrides=config_override)

##Additional Parameter
# overrides = <dict> 
# -example: To set source/target directories
# config_override = dict(
#                        source_dir = "/nfs/jup/sensitivity_classifier/threading/e2e_2023/92Folder",
#                        target_dir = "/nfs/jup/sensitivity_classifier/threading/e2e_data/"
#                       )

In [65]:
param_override = {}

hyperparams=load_hyperparams(overrides=param_override)

##Additional Parameter
# overrides = <dict> 
# -example: 
# param_override = dict(
#                        alpha = 1,
#                        gamma = 1
#                       )

## SeqInt

In [71]:
doc_5w1h_parts = read_parts(config.data5w1h_parts_file)

#### TFIDF

In [72]:
candidate_selection_params={'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

In [73]:
embedding_model="tfidflsa"
use_time_decay= False           #Include Time Decay computation in HAC Linkage
use_entity_similarity = False   #Include Entity Similarity computation in HAC Linkage
minimum_thread_len = 5          #Estimated number of passages in each thread

### Number of clusters(or Threads) ####
    # Use either estimate number of clusters (n_clusters) or the distance_threshold
    # => n_clusters can be set to [Number of Passages]/[Estimated passages in each Thread], e.g. for 1000 passages and 5 passages in each thread, set n_clusters=1000/5=200
    # => distance_threshold can eliminate the need of setting n_clusters parameter. 
    #    In this case number of clusters will be automatically decided based on the distance threshold above which, clusters will not be merged
    #    (higher distance_threshold means less number of clusters)
    
distance_threshold=None                            #None, or should be adjusted according to affinity: euclidean=1.5, cosine=0.8
n_clusters=np.zeros(len(doc_5w1h_parts),int)       #None, or e.g. list of [Number of Passages/Estimated passages in each Thread] for each collection split
for k,v in doc_5w1h_parts.items():
    n_clusters[k]=(v[1]-v[0])//minimum_thread_len

run_seqint(config, hyperparams, emb=embedding_model,n_clusters=n_clusters,distance_threshold=distance_threshold,time_decay=use_time_decay, ent_similarity=use_entity_similarity)


Part: 1
	Ward
	Clustering. Train Data: (399, 200)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_tfidflsa_w0.7_ward_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 6186.98it/s]

Part: 2
	Ward
	Clustering. Train Data: (399, 200)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_tfidflsa_w0.7_ward_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 6383.53it/s]

Part: 3
	Ward
	Clustering. Train Data: (399, 200)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_tfidflsa_w0.7_ward_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 6303.38it/s]

Part: 4
	SKIPPING: n_clusters should be greater than 0. (value provided: 0)
Threads stored at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_tfidflsa_w0.7_ward.p

Thread Count:237
Avg Length:5.050632911392405
Cosine Score: 0.1917581574901042


In [74]:
file_pattern = get_thread_file_pattern("seqint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_seqint_tfidflsa_w0.7_ward.txt




Mean Cosine Similarity (MPDCS): 0.3438
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 444, Thread Sens Docs: 213
Thread Count: 76
Mean Thread Len: 5.8421
Mean Thread days span: 1219.1974
All Sens Threads: 1
Thread Sens Ratios
	 0.00 - 0.25	: Count=9	Mean=0.1000	Std=0.0903
	 0.25 - 0.50	: Count=24	Mean=0.3585	Std=0.0497
	 0.50 - 0.75	: Count=34	Mean=0.5715	Std=0.0704
	 0.75 - 1.00	: Count=9	Mean=0.7889	Std=0.0774


#### TFIDF-TD

In [75]:
#Check Parameter description in the TFIDF cell

embedding_model="tfidflsa"

use_time_decay= True           
use_entity_similarity = False  
minimum_thread_len = 5         
distance_threshold=None                            
n_clusters=np.zeros(len(doc_5w1h_parts),int)
for k,v in doc_5w1h_parts.items():
    n_clusters[k]=(v[1]-v[0])//minimum_thread_len

run_seqint(config, hyperparams, emb=embedding_model,n_clusters=n_clusters,distance_threshold=distance_threshold,time_decay=use_time_decay, ent_similarity=use_entity_similarity)


Part: 1
	Time Decay
	Clustering. Train Data: (399, 201)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_tfidflsa_w0.7_td10_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 6776.21it/s]

Part: 2
	Time Decay
	Clustering. Train Data: (399, 201)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_tfidflsa_w0.7_td10_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 6886.91it/s]

Part: 3
	Time Decay
	Clustering. Train Data: (399, 201)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_tfidflsa_w0.7_td10_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 973.13it/s]

Part: 4
	SKIPPING: n_clusters should be greater than 0. (value provided: 0)
Threads stored at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_tfidflsa_w0.7_td10.p

Thread Count:237
Avg Length:5.050632911392405
Cosine Score: 0.

In [76]:
file_pattern = get_thread_file_pattern("seqint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_seqint_tfidflsa_w0.7_td10.txt




Mean Cosine Similarity (MPDCS): 0.3369
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 26, Thread Sens Docs: 12
Thread Count: 8
Mean Thread Len: 3.2500
Mean Thread days span: 79.6250
All Sens Threads: 1
Thread Sens Ratios
	 0.00 - 0.25	: Count=1	Mean=0.0000	Std=0.0000
	 0.25 - 0.50	: Count=4	Mean=0.3125	Std=0.0361
	 0.50 - 0.75	: Count=2	Mean=0.6667	Std=0.0000
	 0.75 - 1.00	: Count=1	Mean=1.0000	Std=0.0000


#### TFIDF-TD-ENT

In [77]:
#Check Parameter description in the TFIDF cell

embedding_model="tfidflsa"
use_time_decay= True
use_entity_similarity = True
minimum_thread_len = 5
distance_threshold=None
n_clusters=np.zeros(len(doc_5w1h_parts),int)
for k,v in doc_5w1h_parts.items():
    n_clusters[k]=(v[1]-v[0])//minimum_thread_len

run_seqint(config, hyperparams, emb=embedding_model,n_clusters=n_clusters,distance_threshold=distance_threshold,time_decay=use_time_decay, ent_similarity=use_entity_similarity)


Part: 1
Entity Graph: Number of Nodes= 427. Number of Edges= 31
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_tfidflsa_w0.7_td10_ent0.1_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 6774.69it/s]

Part: 2
Entity Graph: Number of Nodes= 417. Number of Edges= 32
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_tfidflsa_w0.7_td10_ent0.1_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 6894.94it/s]

Part: 3
Entity Graph: Number of Nodes= 418. Number of Edges= 22
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_tfidflsa_w0.7_td10_ent0.1_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 6854.29i

In [78]:
file_pattern = get_thread_file_pattern("seqint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_seqint_tfidflsa_w0.7_td10_ent0.1.txt




Mean Cosine Similarity (MPDCS): 0.3310
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 29, Thread Sens Docs: 13
Thread Count: 9
Mean Thread Len: 3.2222
Mean Thread days span: 78.3333
All Sens Threads: 1
Thread Sens Ratios
	 0.00 - 0.25	: Count=1	Mean=0.0000	Std=0.0000
	 0.25 - 0.50	: Count=5	Mean=0.3167	Std=0.0333
	 0.50 - 0.75	: Count=2	Mean=0.6667	Std=0.0000
	 0.75 - 1.00	: Count=1	Mean=1.0000	Std=0.0000


#### MiniLM

In [79]:
#Check Parameter description in the TFIDF cell

embedding_model="minilm"
use_time_decay= False
use_entity_similarity = False
minimum_thread_len = 5
distance_threshold=None
n_clusters=np.zeros(len(doc_5w1h_parts),int)
for k,v in doc_5w1h_parts.items():
    n_clusters[k]=(v[1]-v[0])//minimum_thread_len

run_seqint(config, hyperparams, emb=embedding_model,n_clusters=n_clusters,distance_threshold=distance_threshold,time_decay=use_time_decay, ent_similarity=use_entity_similarity)


Part: 1
	Ward
	Clustering. Train Data: (399, 384)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_minilm_w0.7_ward_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 4835.89it/s]

Part: 2
	Ward
	Clustering. Train Data: (399, 384)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_minilm_w0.7_ward_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 4847.91it/s]

Part: 3
	Ward
	Clustering. Train Data: (399, 384)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_minilm_w0.7_ward_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 4890.78it/s]

Part: 4
	SKIPPING: n_clusters should be greater than 0. (value provided: 0)
Threads stored at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_minilm_w0.7_ward.p

Thread Count:237
Avg Length:5.050632911392405
Cosine Score: 0.4420757817993689


In [80]:
file_pattern = get_thread_file_pattern("seqint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_seqint_minilm_w0.7_ward.txt




Mean Cosine Similarity (MPDCS): 0.4456
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 1130, Thread Sens Docs: 558
Thread Count: 204
Mean Thread Len: 5.5392
Mean Thread days span: 1204.6422
All Sens Threads: 9
Thread Sens Ratios
	 0.00 - 0.25	: Count=30	Mean=0.0718	Std=0.0897
	 0.25 - 0.50	: Count=60	Mean=0.3482	Std=0.0583
	 0.50 - 0.75	: Count=82	Mean=0.5969	Std=0.0710
	 0.75 - 1.00	: Count=32	Mean=0.8337	Std=0.1059


#### MiniLM-TD

In [81]:
#Check Parameter description in the TFIDF cell

embedding_model="minilm"
use_time_decay= True
use_entity_similarity = False
minimum_thread_len = 5
distance_threshold=None
n_clusters=np.zeros(len(doc_5w1h_parts),int)
for k,v in doc_5w1h_parts.items():
    n_clusters[k]=(v[1]-v[0])//minimum_thread_len

run_seqint(config, hyperparams, emb=embedding_model,n_clusters=n_clusters,distance_threshold=distance_threshold,time_decay=use_time_decay, ent_similarity=use_entity_similarity)


Part: 1
	Time Decay
	Clustering. Train Data: (399, 385)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_minilm_w0.7_td10_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 760.42it/s]

Part: 2
	Time Decay
	Clustering. Train Data: (399, 385)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_minilm_w0.7_td10_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 828.12it/s]

Part: 3
	Time Decay
	Clustering. Train Data: (399, 385)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_minilm_w0.7_td10_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 879.99it/s]

Part: 4
	SKIPPING: n_clusters should be greater than 0. (value provided: 0)
Threads stored at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_minilm_w0.7_td10.p

Thread Count:237
Avg Length:5.050632911392405
Cosine Score: 0.5158564874

In [82]:
file_pattern = get_thread_file_pattern("seqint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_seqint_minilm_w0.7_td10.txt




Mean Cosine Similarity (MPDCS): 0.5626
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 96, Thread Sens Docs: 53
Thread Count: 26
Mean Thread Len: 3.6923
Mean Thread days span: 37.6538
All Sens Threads: 3
Thread Sens Ratios
	 0.00 - 0.25	: Count=2	Mean=0.0000	Std=0.0000
	 0.25 - 0.50	: Count=8	Mean=0.3229	Std=0.0276
	 0.50 - 0.75	: Count=9	Mean=0.6296	Std=0.0693
	 0.75 - 1.00	: Count=7	Mean=0.8571	Std=0.1237


#### MiniLM-TD-ENT

In [83]:
#Check Parameter description in the TFIDF cell

embedding_model="minilm"
use_time_decay= True
use_entity_similarity = True
minimum_thread_len = 5
distance_threshold=None
n_clusters=np.zeros(len(doc_5w1h_parts),int)
for k,v in doc_5w1h_parts.items():
    n_clusters[k]=(v[1]-v[0])//minimum_thread_len

run_seqint(config, hyperparams, emb=embedding_model,n_clusters=n_clusters,distance_threshold=distance_threshold,time_decay=use_time_decay, ent_similarity=use_entity_similarity)


Part: 1
Entity Graph: Number of Nodes= 427. Number of Edges= 31
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 386)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_minilm_w0.7_td10_ent0.1_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5300.41it/s]

Part: 2
Entity Graph: Number of Nodes= 417. Number of Edges= 32
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 386)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_minilm_w0.7_td10_ent0.1_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5164.35it/s]

Part: 3
Entity Graph: Number of Nodes= 418. Number of Edges= 22
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 386)
	Cluster Found: 79
	Model Dumped at: /nfs/jup/sensitivity_classifier/threading/e2e_data/seqint/hac_minilm_w0.7_td10_ent0.1_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5323.75it/s]



In [84]:
file_pattern = get_thread_file_pattern("seqint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_seqint_minilm_w0.7_td10_ent0.1.txt




Mean Cosine Similarity (MPDCS): 0.5626
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 96, Thread Sens Docs: 53
Thread Count: 26
Mean Thread Len: 3.6923
Mean Thread days span: 37.6538
All Sens Threads: 3
Thread Sens Ratios
	 0.00 - 0.25	: Count=2	Mean=0.0000	Std=0.0000
	 0.25 - 0.50	: Count=8	Mean=0.3229	Std=0.0276
	 0.50 - 0.75	: Count=9	Mean=0.6296	Std=0.0693
	 0.75 - 1.00	: Count=7	Mean=0.8571	Std=0.1237


#### Roberta

Only Run if the Roberta embeddings are computed 

vectorize_5w1h(destPath,nltk_dir=nltk_dir,transformer_dir=transformer_dir,vect_roberta=True) <br>
or <br>
vectorize_5w1h(destPath,nltk_dir=nltk_dir,transformer_dir=transformer_dir,vect_roberta=True,vect_tfidf=False,vect_minilm=False)

In [ ]:
#Check Parameter description in the TFIDF cell

embedding_model="roberta"
use_time_decay= False
use_entity_similarity = False
minimum_thread_len = 5
distance_threshold=None
n_clusters=np.zeros(len(doc_5w1h_parts),int)
for k,v in doc_5w1h_parts.items():
    n_clusters[k]=(v[1]-v[0])//minimum_thread_len

run_seqint(config, hyperparams, emb=embedding_model,n_clusters=n_clusters,distance_threshold=distance_threshold,time_decay=use_time_decay, ent_similarity=use_entity_similarity)

In [ ]:
file_pattern = get_thread_file_pattern("seqint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

#### Roberta-TD

In [ ]:
#Check Parameter description in the TFIDF cell

embedding_model="roberta"
use_time_decay= True
use_entity_similarity = False
minimum_thread_len = 5
distance_threshold=None
n_clusters=np.zeros(len(doc_5w1h_parts),int)
for k,v in doc_5w1h_parts.items():
    n_clusters[k]=(v[1]-v[0])//minimum_thread_len

run_seqint(config, hyperparams, emb=embedding_model,n_clusters=n_clusters,distance_threshold=distance_threshold,time_decay=use_time_decay, ent_similarity=use_entity_similarity)

In [148]:
file_pattern = get_thread_file_pattern("seqint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [0, 10000], 'similarity': [0, 100], 'days_num': [0, 5000]}

Results Saved at: /app/DocumentSets/threads/eval_seqint_tfidflsa_w0.7_td10_ent0.1.txt




Mean Cosine Similarity (MPDCS): 0.2852
Total Docs: 1203, Total Sens Docs: 614
Thread Docs: 1022, Thread Sens Docs: 532
Thread Count: 62
Mean Thread Len: 16.4839
Mean Thread days span: 117.1613
All Sens Threads: 18
Thread Sens Ratios
	 0.00 - 0.25	: Count=15	Mean=0.0000	Std=0.0000
	 0.25 - 0.50	: Count=2	Mean=0.3333	Std=0.0000
	 0.50 - 0.75	: Count=27	Mean=0.5083	Std=0.0319
	 0.75 - 1.00	: Count=18	Mean=1.0000	Std=0.0000


#### Roberta-TD-ENT

In [151]:
#Check Parameter description in the TFIDF cell

embedding_model="roberta"
use_time_decay= True
use_entity_similarity = True
minimum_thread_len = 5
distance_threshold=None
n_clusters=np.zeros(len(doc_5w1h_parts),int)
for k,v in doc_5w1h_parts.items():
    n_clusters[k]=(v[1]-v[0])//minimum_thread_len

run_seqint(config, hyperparams, emb=embedding_model,n_clusters=n_clusters,distance_threshold=distance_threshold,time_decay=use_time_decay, ent_similarity=use_entity_similarity)


Part: 1
Entity Graph: Number of Nodes= 427. Number of Edges= 31
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5347.55it/s]

Part: 2
Entity Graph: Number of Nodes= 417. Number of Edges= 32
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5316.66it/s]

Part: 3
Entity Graph: Number of Nodes= 418. Number of Edges= 22
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5679.25it/s]

Part: 4
	SKIPPING: n_clusters should be greater than 0. (value provided: 0)
Threads stored at

In [148]:
file_pattern = get_thread_file_pattern("seqint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [0, 10000], 'similarity': [0, 100], 'days_num': [0, 5000]}

Results Saved at: /app/DocumentSets/threads/eval_seqint_tfidflsa_w0.7_td10_ent0.1.txt




Mean Cosine Similarity (MPDCS): 0.2852
Total Docs: 1203, Total Sens Docs: 614
Thread Docs: 1022, Thread Sens Docs: 532
Thread Count: 62
Mean Thread Len: 16.4839
Mean Thread days span: 117.1613
All Sens Threads: 18
Thread Sens Ratios
	 0.00 - 0.25	: Count=15	Mean=0.0000	Std=0.0000
	 0.25 - 0.50	: Count=2	Mean=0.3333	Std=0.0000
	 0.50 - 0.75	: Count=27	Mean=0.5083	Std=0.0319
	 0.75 - 1.00	: Count=18	Mean=1.0000	Std=0.0000


## HINT

In [5]:
doc_5w1h_parts = read_parts(config.data5w1h_parts_file)

In [8]:
## Test cupy
import cupy as cp
_ = cp.dot(cp.ones((4,2)),cp.ones((4,2)).T)

array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]])

#### TFIDF

In [11]:
candidate_selection_params={'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

In [9]:
embedding_model="tfidflsa"
use_time_decay= False           #Include Time Decay computation in HAC Linkage
use_entity_similarity = False   #Include Entity Similarity computation in HAC Linkage
use_gpu = True

### Number of clusters(or Threads) ####
    # Use either estimate number of clusters (n_clusters) or the distance_threshold
    # => n_clusters can be set to [Number of Passages]/[Estimated passages in each Thread], e.g. for 1000 passages and 5 passages in each thread, set n_clusters=1000/5=200
    # => distance_threshold can eliminate the need of setting n_clusters parameter. 
    #    In this case number of clusters will be automatically decided based on the distance threshold above which, clusters will not be merged
    #    (higher distance_threshold means less number of clusters)

    
# help='method name: npc, samp, louvain, leiden'    
    
run_hint(config, hyperparams, emb=embedding_model,time_decay=use_time_decay, ent_similarity=use_entity_similarity,use_gpu=use_gpu)

Part: 1
	Train Data: (399, 200). TD: False, ENT: False, Continous: False
	Creating Graph
Number of Edges: 1190: 100%|██████████| 399/399 [00:00<00:00, 959.64it/s]
	Running NPC
/nfs/jup/sensitivity_classifier/threading/e2e_2023/sourceCode/hint/npc.py:39: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph).tocsc()
	Generating Threads
/root/miniconda/lib/python3.8/site-packages/numpy/core/numeric.py:492: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return multiarray.count_nonzero(a)
100%|██████████| 109/109 [00:00<00:00, 12979.93it/s]
Part: 2
	Train Data: (399, 200). TD: False, ENT: False, Continous: False
	Creating Graph
Number of Edges: 4267: 100%|██████████| 399/399 [00:00<00:00, 28331

In [12]:
file_pattern = get_thread_file_pattern("hint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_hint_npc_tfidflsa_w0.7.txt




Mean Cosine Similarity (MPDCS): 0.3299
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 453, Thread Sens Docs: 211
Thread Count: 88
Mean Thread Len: 5.1477
Mean Thread days span: 1230.2500
All Sens Threads: 4
Thread Sens Ratios
	 0.00 - 0.25	: Count=14	Mean=0.0959	Std=0.0970
	 0.25 - 0.50	: Count=27	Mean=0.3385	Std=0.0542
	 0.50 - 0.75	: Count=33	Mean=0.5886	Std=0.0713
	 0.75 - 1.00	: Count=14	Mean=0.8345	Std=0.1075


#### TFIDF-TD

In [13]:
#Check Parameter description in the TFIDF cell

embedding_model="tfidflsa"

use_time_decay= True           
use_entity_similarity = False  
use_gpu = True

run_hint(config, hyperparams, emb=embedding_model,time_decay=use_time_decay, ent_similarity=use_entity_similarity,use_gpu=use_gpu)

Part: 1
	Train Data: (399, 201). TD: True, ENT: False, Continous: False
	Creating Graph
Number of Edges: 62: 100%|██████████| 399/399 [00:01<00:00, 370.70it/s]
	Running NPC
/nfs/jup/sensitivity_classifier/threading/e2e_2023/sourceCode/hint/npc.py:39: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph).tocsc()
	Generating Threads
/root/miniconda/lib/python3.8/site-packages/numpy/core/numeric.py:492: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return multiarray.count_nonzero(a)
100%|██████████| 49/49 [00:00<00:00, 21861.60it/s]
Part: 2
	Train Data: (399, 201). TD: True, ENT: False, Continous: False
	Creating Graph
Number of Edges: 303: 100%|██████████| 399/399 [00:00<00:00, 51992.27it/s

In [14]:
file_pattern = get_thread_file_pattern("hint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_hint_npc_tfidflsa_w0.7_td10.txt




Mean Cosine Similarity (MPDCS): 0.3635
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 157, Thread Sens Docs: 77
Thread Count: 38
Mean Thread Len: 4.1316
Mean Thread days span: 81.1842
All Sens Threads: 4
Thread Sens Ratios
	 0.00 - 0.25	: Count=6	Mean=0.0944	Std=0.0951
	 0.25 - 0.50	: Count=10	Mean=0.3233	Std=0.0416
	 0.50 - 0.75	: Count=16	Mean=0.5767	Std=0.0753
	 0.75 - 1.00	: Count=6	Mean=0.9250	Std=0.1070


#### TFIDF-TD-ENT

In [15]:
#Check Parameter description in the TFIDF cell

embedding_model="tfidflsa"
use_time_decay= True
use_entity_similarity = True
use_gpu = True

run_hint(config, hyperparams, emb=embedding_model,time_decay=use_time_decay, ent_similarity=use_entity_similarity,use_gpu=use_gpu)

Part: 1
	Entity Graph: Number of Nodes= 427. Number of Edges= 31
	Train Data: (399, 201). TD: True, ENT: True, Continous: False
	Creating Graph
Number of Edges: 62: 100%|██████████| 399/399 [00:00<00:00, 37480.18it/s]
	Running NPC
/nfs/jup/sensitivity_classifier/threading/e2e_2023/sourceCode/hint/npc.py:39: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph).tocsc()
	Generating Threads
/root/miniconda/lib/python3.8/site-packages/numpy/core/numeric.py:492: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return multiarray.count_nonzero(a)
100%|██████████| 49/49 [00:00<00:00, 21875.56it/s]
Part: 2
	Entity Graph: Number of Nodes= 417. Number of Edges= 32
	Train Data: (399, 201). TD: True, ENT

In [16]:
file_pattern = get_thread_file_pattern("hint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_hint_npc_tfidflsa_w0.7_td10_ent0.1.txt




Mean Cosine Similarity (MPDCS): 0.3635
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 157, Thread Sens Docs: 77
Thread Count: 38
Mean Thread Len: 4.1316
Mean Thread days span: 81.1842
All Sens Threads: 4
Thread Sens Ratios
	 0.00 - 0.25	: Count=6	Mean=0.0944	Std=0.0951
	 0.25 - 0.50	: Count=10	Mean=0.3233	Std=0.0416
	 0.50 - 0.75	: Count=16	Mean=0.5767	Std=0.0753
	 0.75 - 1.00	: Count=6	Mean=0.9250	Std=0.1070


#### MiniLM

In [17]:
#Check Parameter description in the TFIDF cell

embedding_model="minilm"
use_time_decay= False
use_entity_similarity = False
use_gpu = True

run_hint(config, hyperparams, emb=embedding_model,time_decay=use_time_decay, ent_similarity=use_entity_similarity,use_gpu=use_gpu)

Part: 1
	Train Data: (399, 384). TD: False, ENT: False, Continous: False
	Creating Graph
Number of Edges: 4946: 100%|██████████| 399/399 [00:00<00:00, 954.40it/s]
	Running NPC
/nfs/jup/sensitivity_classifier/threading/e2e_2023/sourceCode/hint/npc.py:39: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph).tocsc()
	Generating Threads
/root/miniconda/lib/python3.8/site-packages/numpy/core/numeric.py:492: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return multiarray.count_nonzero(a)
100%|██████████| 101/101 [00:00<00:00, 8981.38it/s]
Part: 2
	Train Data: (399, 384). TD: False, ENT: False, Continous: False
	Creating Graph
Number of Edges: 15413: 100%|██████████| 399/399 [00:00<00:00, 12756

In [18]:
file_pattern = get_thread_file_pattern("hint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_hint_npc_minilm_w0.7.txt




Mean Cosine Similarity (MPDCS): 0.4543
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 1061, Thread Sens Docs: 514
Thread Count: 209
Mean Thread Len: 5.0766
Mean Thread days span: 1115.7177
All Sens Threads: 8
Thread Sens Ratios
	 0.00 - 0.25	: Count=24	Mean=0.0731	Std=0.0950
	 0.25 - 0.50	: Count=84	Mean=0.3406	Std=0.0521
	 0.50 - 0.75	: Count=68	Mean=0.5902	Std=0.0698
	 0.75 - 1.00	: Count=33	Mean=0.8398	Std=0.0957


#### MiniLM-TD

In [19]:
#Check Parameter description in the TFIDF cell

embedding_model="minilm"
use_time_decay= True
use_entity_similarity = False
use_gpu = True

run_hint(config, hyperparams, emb=embedding_model,time_decay=use_time_decay, ent_similarity=use_entity_similarity,use_gpu=use_gpu)

Part: 1
	Train Data: (399, 385). TD: True, ENT: False, Continous: False
	Creating Graph
Number of Edges: 182: 100%|██████████| 399/399 [00:00<00:00, 51406.15it/s]
	Running NPC
/nfs/jup/sensitivity_classifier/threading/e2e_2023/sourceCode/hint/npc.py:39: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph).tocsc()
	Generating Threads
/root/miniconda/lib/python3.8/site-packages/numpy/core/numeric.py:492: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return multiarray.count_nonzero(a)
100%|██████████| 74/74 [00:00<00:00, 15431.74it/s]
Part: 2
	Train Data: (399, 385). TD: True, ENT: False, Continous: False
	Creating Graph
Number of Edges: 949: 100%|██████████| 399/399 [00:00<00:00, 42548.75i

In [20]:
file_pattern = get_thread_file_pattern("hint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_hint_npc_minilm_w0.7_td10.txt




Mean Cosine Similarity (MPDCS): 0.4843
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 488, Thread Sens Docs: 232
Thread Count: 118
Mean Thread Len: 4.1356
Mean Thread days span: 59.8983
All Sens Threads: 8
Thread Sens Ratios
	 0.00 - 0.25	: Count=16	Mean=0.0604	Std=0.0899
	 0.25 - 0.50	: Count=41	Mean=0.3297	Std=0.0443
	 0.50 - 0.75	: Count=43	Mean=0.6023	Std=0.0717
	 0.75 - 1.00	: Count=18	Mean=0.8694	Std=0.1180


#### MiniLM-TD-ENT

In [21]:
#Check Parameter description in the TFIDF cell

embedding_model="minilm"
use_time_decay= True
use_entity_similarity = True
use_gpu = True

run_hint(config, hyperparams, emb=embedding_model,time_decay=use_time_decay, ent_similarity=use_entity_similarity,use_gpu=use_gpu)

Part: 1
	Entity Graph: Number of Nodes= 427. Number of Edges= 31
	Train Data: (399, 385). TD: True, ENT: True, Continous: False
	Creating Graph
Number of Edges: 182: 100%|██████████| 399/399 [00:00<00:00, 27777.31it/s]
	Running NPC
/nfs/jup/sensitivity_classifier/threading/e2e_2023/sourceCode/hint/npc.py:39: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph).tocsc()
	Generating Threads
/root/miniconda/lib/python3.8/site-packages/numpy/core/numeric.py:492: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return multiarray.count_nonzero(a)
100%|██████████| 74/74 [00:00<00:00, 15473.28it/s]
Part: 2
	Entity Graph: Number of Nodes= 417. Number of Edges= 32
	Train Data: (399, 385). TD: True, EN

In [22]:
file_pattern = get_thread_file_pattern("hint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [3, 100], 'similarity': [0.2, 0.8], 'days_num': [0, 2000]}

Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_hint_npc_minilm_w0.7_td10_ent0.1.txt




Mean Cosine Similarity (MPDCS): 0.4844
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 487, Thread Sens Docs: 231
Thread Count: 118
Mean Thread Len: 4.1271
Mean Thread days span: 59.2712
All Sens Threads: 8
Thread Sens Ratios
	 0.00 - 0.25	: Count=16	Mean=0.0604	Std=0.0899
	 0.25 - 0.50	: Count=41	Mean=0.3297	Std=0.0443
	 0.50 - 0.75	: Count=44	Mean=0.6037	Std=0.0715
	 0.75 - 1.00	: Count=17	Mean=0.8765	Std=0.1177


#### Roberta

Only Run if the Roberta embeddings are computed 

vectorize_5w1h(destPath,nltk_dir=nltk_dir,transformer_dir=transformer_dir,vect_roberta=True) <br>
or <br>
vectorize_5w1h(destPath,nltk_dir=nltk_dir,transformer_dir=transformer_dir,vect_roberta=True,vect_tfidf=False,vect_minilm=False)

In [151]:
#Check Parameter description in the TFIDF cell

embedding_model="roberta"
use_time_decay= False
use_entity_similarity = False
use_gpu = True

run_hint(config, hyperparams, emb=embedding_model,time_decay=use_time_decay, ent_similarity=use_entity_similarity,use_gpu=use_gpu)


Part: 1
Entity Graph: Number of Nodes= 427. Number of Edges= 31
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5347.55it/s]

Part: 2
Entity Graph: Number of Nodes= 417. Number of Edges= 32
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5316.66it/s]

Part: 3
Entity Graph: Number of Nodes= 418. Number of Edges= 22
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5679.25it/s]

Part: 4
	SKIPPING: n_clusters should be greater than 0. (value provided: 0)
Threads stored at

In [148]:
file_pattern = get_thread_file_pattern("hint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [0, 10000], 'similarity': [0, 100], 'days_num': [0, 5000]}

Results Saved at: /app/DocumentSets/threads/eval_seqint_tfidflsa_w0.7_td10_ent0.1.txt




Mean Cosine Similarity (MPDCS): 0.2852
Total Docs: 1203, Total Sens Docs: 614
Thread Docs: 1022, Thread Sens Docs: 532
Thread Count: 62
Mean Thread Len: 16.4839
Mean Thread days span: 117.1613
All Sens Threads: 18
Thread Sens Ratios
	 0.00 - 0.25	: Count=15	Mean=0.0000	Std=0.0000
	 0.25 - 0.50	: Count=2	Mean=0.3333	Std=0.0000
	 0.50 - 0.75	: Count=27	Mean=0.5083	Std=0.0319
	 0.75 - 1.00	: Count=18	Mean=1.0000	Std=0.0000


#### Roberta-TD

In [151]:
#Check Parameter description in the TFIDF cell

embedding_model="roberta"
use_time_decay= True
use_entity_similarity = False
use_gpu = True

run_hint(config, hyperparams, emb=embedding_model,time_decay=use_time_decay, ent_similarity=use_entity_similarity,use_gpu=use_gpu)


Part: 1
Entity Graph: Number of Nodes= 427. Number of Edges= 31
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5347.55it/s]

Part: 2
Entity Graph: Number of Nodes= 417. Number of Edges= 32
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5316.66it/s]

Part: 3
Entity Graph: Number of Nodes= 418. Number of Edges= 22
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5679.25it/s]

Part: 4
	SKIPPING: n_clusters should be greater than 0. (value provided: 0)
Threads stored at

In [148]:
file_pattern = get_thread_file_pattern("hint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [0, 10000], 'similarity': [0, 100], 'days_num': [0, 5000]}

Results Saved at: /app/DocumentSets/threads/eval_seqint_tfidflsa_w0.7_td10_ent0.1.txt




Mean Cosine Similarity (MPDCS): 0.2852
Total Docs: 1203, Total Sens Docs: 614
Thread Docs: 1022, Thread Sens Docs: 532
Thread Count: 62
Mean Thread Len: 16.4839
Mean Thread days span: 117.1613
All Sens Threads: 18
Thread Sens Ratios
	 0.00 - 0.25	: Count=15	Mean=0.0000	Std=0.0000
	 0.25 - 0.50	: Count=2	Mean=0.3333	Std=0.0000
	 0.50 - 0.75	: Count=27	Mean=0.5083	Std=0.0319
	 0.75 - 1.00	: Count=18	Mean=1.0000	Std=0.0000


#### Roberta-TD-ENT

In [151]:
#Check Parameter description in the TFIDF cell

embedding_model="roberta"
use_time_decay= True
use_entity_similarity = True
use_gpu = True

run_hint(config, hyperparams, emb=embedding_model,time_decay=use_time_decay, ent_similarity=use_entity_similarity,use_gpu=use_gpu)


Part: 1
Entity Graph: Number of Nodes= 427. Number of Edges= 31
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_0.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5347.55it/s]

Part: 2
Entity Graph: Number of Nodes= 417. Number of Edges= 32
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_1.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5316.66it/s]

Part: 3
Entity Graph: Number of Nodes= 418. Number of Edges= 22
	Time Decay with Entity Similarity
	Clustering. Train Data: (399, 202)
	Cluster Found: 79
	Model Dumped at: /app/DocumentSets/seqint/hac_tfidflsa_w0.7_td10_ent0.1_2.p
	Generating Threads
100%|██████████| 79/79 [00:00<00:00, 5679.25it/s]

Part: 4
	SKIPPING: n_clusters should be greater than 0. (value provided: 0)
Threads stored at

In [148]:
file_pattern = get_thread_file_pattern("hint",emb=embedding_model,hyperparams=hyperparams,time_decay=use_time_decay,ent_similarity=use_entity_similarity)
_ = eval_filter_threads(config, file_pattern,params=candidate_selection_params)

Filtering using the following parameters:
{'thread_len': [0, 10000], 'similarity': [0, 100], 'days_num': [0, 5000]}

Results Saved at: /app/DocumentSets/threads/eval_seqint_tfidflsa_w0.7_td10_ent0.1.txt




Mean Cosine Similarity (MPDCS): 0.2852
Total Docs: 1203, Total Sens Docs: 614
Thread Docs: 1022, Thread Sens Docs: 532
Thread Count: 62
Mean Thread Len: 16.4839
Mean Thread days span: 117.1613
All Sens Threads: 18
Thread Sens Ratios
	 0.00 - 0.25	: Count=15	Mean=0.0000	Std=0.0000
	 0.25 - 0.50	: Count=2	Mean=0.3333	Std=0.0000
	 0.50 - 0.75	: Count=27	Mean=0.5083	Std=0.0319
	 0.75 - 1.00	: Count=18	Mean=1.0000	Std=0.0000


## KSDPP

In [7]:
compute_passage_cosine(config,100)

====Part#1====
100%|██████████| 400/400 [00:00<00:00, 1050.87it/s]
====Part#2====
100%|██████████| 400/400 [00:00<00:00, 2275.00it/s]
====Part#3====
100%|██████████| 400/400 [00:00<00:00, 646.91it/s]
====Part#4====
400it [00:00, 2519.12it/s]           


In [8]:
create_graphs(config, weight_threshold=0.2, cos_weight_batch_size=100)

====Part#1====
N:400, E:1680: 100%|██████████| 4/4 [00:00<00:00, 125.40it/s]
/nfs/jup/sensitivity_classifier/threading/e2e_2023/sourceCode/dpp/dpp_preprocess.py:137: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj=nx.adjacency_matrix(DG,nodelist=sorted(list(DG.nodes)),weight="weight")
====Part#2====
N:400, E:508: 100%|██████████| 4/4 [00:00<00:00, 39.20it/s] 
/nfs/jup/sensitivity_classifier/threading/e2e_2023/sourceCode/dpp/dpp_preprocess.py:137: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj=nx.adjacency_matrix(DG,nodelist=sorted(list(DG.nodes)),weight="weight")
====Part#3====
N:400, E:111: 100%|██████████| 4/4 [00:00<00:00, 133.72it/s]
/nfs/jup/sensitivity_classifier/threading/e2e_2023/sourceCode/dpp/dpp_preprocess.py:137: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj=nx.adjacency_matrix(DG,nodelist=sort

In [9]:
convert_to_mat(config)

====Part#1====
Q1=(400,)
G=(400, 51)
A=(400, 400)
====Part#2====
Q1=(400,)
G=(400, 51)
/root/miniconda/lib/python3.8/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
A=(400, 400)
====Part#3====
Q1=(400,)
G=(400, 51)
A=(400, 400)
====Part#4====
	SKIPPING: Graph not found: /nfs/jup/sensitivity_classifier/threading/e2e_data/dpp/graph_3.gp.


In [44]:
#In case of any error change the following parameters in  /sourceCode/dpp/examples/ksdpp_thread.m
# [ln5]=> T = 5; %  Number of Passages in a thread, try with a lower value such as 2 or 3
# [ln8]=> k = 20; %  Number of Threads to sample in an epoch, try with a lower value such as 5 or 10

run_ksdpp(config,epochs=100)  #Runs a Matlab code in background to sample threads

====Part#1====
Total: 2000, Error:0: 100%|██████████| 100/100 [00:14<00:00,  6.67it/s]
====Part#2====
Total: 0, Error:0:   0%|          | 0/100 [00:00<?, ?it/s]Unable to perform assignment because the left and right sides have a different number of elements.

Error in bp (line 272)
        Y(t) = find(rand <= cumsum(dist) / sum(dist),1);

Error in sample_sdpp (line 27)
  Y(:,i) = bp(model,'sample2',V);

Error in ksdpp_thread (line 31)
sdpp_sample = sample_sdpp(model,C,k);

Total: 20, Error:1:   1%|          | 1/100 [00:00<01:12,  1.36it/s]Unable to perform assignment because the left and right sides have a different number of elements.

Error in bp (line 272)
        Y(t) = find(rand <= cumsum(dist) / sum(dist),1);

Error in sample_sdpp (line 27)
  Y(:,i) = bp(model,'sample2',V);

Error in ksdpp_thread (line 31)
sdpp_sample = sample_sdpp(model,C,k);

Total: 60, Error:2:   3%|▎         | 3/100 [00:01<01:00,  1.62it/s]Unable to perform assignment because the left and right sides have a d

In [67]:
_ = eval_sdpp_threads(config, "dpp")



Mean Cosine Similarity (MPDCS): 0.2939
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 112, Thread Sens Docs: 45
Thread Count: 40
Mean Thread Len: 2.8000
Mean Thread days span: 318.0250
All Sens Threads: 5
Thread Sens Ratios
	 0.00 - 0.25	: Count=13	Mean=0.0615	Std=0.0923
	 0.25 - 0.50	: Count=2	Mean=0.2917	Std=0.0417
	 0.50 - 0.75	: Count=20	Mean=0.5267	Std=0.0554
	 0.75 - 1.00	: Count=5	Mean=1.0000	Std=0.0000



Results Saved at: /nfs/jup/sensitivity_classifier/threading/e2e_data/threads/eval_dpp.txt


## Print Threads

In [23]:
paragraphDF = pd.DataFrame(read_json_dump(config.passage_dict_file)).set_index("doc_id")
paragraphDF["created"] = pd.to_datetime(paragraphDF["created"])
data_5w1h=read_dict_dump(config.data5w1h_dict_file, compress=True)

In [24]:
print("Thread Files:")
for res_file in glob.glob(os.path.join(config.threads_dir,f'threads*.p')):
    print("\t"+res_file)

Thread Files:
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_dpp.p
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_tfidflsa_w0.7_ward.p
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_tfidflsa_w0.7_td10.p
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_tfidflsa_w0.7_td10_ent0.1.p
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_minilm_w0.7_ward.p
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_minilm_w0.7_td10.p
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_seqint_minilm_w0.7_td10_ent0.1.p
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_hint_npc_tfidflsa_w0.7.p
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_hint_npc_tfidflsa_w0.7_td10.p
	/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_hint_npc_tfidflsa_w0.7_td10_ent0.1.p
	/nfs/jup/sensitivity_classifier

##### Select a file from the above list to find threads
##### e.g. Roberta-TD

In [25]:
thread_file="/nfs/jup/sensitivity_classifier/threading/e2e_data/threads/threads_hint_npc_minilm_w0.7_td10_ent0.1.p"  #Change this url based on the above list
threads=read_thread_file(thread_file)

In [39]:
keyword_df,thread_df,keyword2thread=get_threads_keywords(config,"hint_minilm_td_ent",threads,top_n=3,force=True)

print(f"Total Keywords: {len(keyword_df)}")
print(f"Total Threads: {len(thread_df)}")

100%|██████████| 267/267 [00:00<00:00, 1284.07it/s]
Keywords weights saved to : /nfs/jup/sensitivity_classifier/threading/e2e_data/sampling/hint_minilm_td_ent_keywords.json.gz
Keywords2Thread dictionary saved to : /nfs/jup/sensitivity_classifier/threading/e2e_data/sampling/hint_minilm_td_ent_keyword2thread.json.gz
Thread weights saved to : /nfs/jup/sensitivity_classifier/threading/e2e_data/sampling/hint_minilm_td_ent_threads.json.gz


Total Keywords: 580
Total Threads: 267


In [40]:
keyword_df.head()

,threads,mean_weight
word,,
happy,1,-14.816993
james,1,-14.829668
called,1,-14.869827
fenwick,1,-14.869827
online,1,-14.883680


In [41]:
thread_df.head()

,length,period,keywords,keyword_weight
137,4,114 days,"[group, change, distribution]",-14.833941
111,5,51 days,"[group, distribution, aware]",-14.838421
95,4,70 days,"[copy, could, grateful]",-14.853036
122,4,35 days,"[called, fenwick, distribution]",-14.871117
90,3,66 days,"[copy, would, pass]",-14.893175


In [43]:
sampled_thread_df=filter_threads(thread_df,min_length=3,max_length=8, min_days=1,max_days=365)
print(f"Total Sampled Threads: {len(sampled_thread_df)}")
sampled_thread_df.head()

Total Sampled Threads: 115


,length,period,keywords,keyword_weight
137,4,114 days,"[group, change, distribution]",-14.833941
111,5,51 days,"[group, distribution, aware]",-14.838421
95,4,70 days,"[copy, could, grateful]",-14.853036
122,4,35 days,"[called, fenwick, distribution]",-14.871117
90,3,66 days,"[copy, would, pass]",-14.893175


#### Print thread from thread_df index

In [44]:
## Args ##
# => thread_idx: <int> Index of the thread to print from thread_df (or threads)
# => show_answers: <boolean> Show/Hide 5W1H extracts
# => outside_text: <boolean> Show 5W1H extracts outside passage (True) or as highlight the extracts within the passage (False)
#                   Set True if the output is not displayed properly or the see all 5w1H answers together

thread_idx=82
print_thread(threads,thread_idx,paragraphDF,data_5w1h)#,outside_text=True)

Thread length: 5
Period: 1999-04-14 00:00:00 to 1999-09-13 00:00:00

1) doc262#wc_1 1999-04-14 00:00:00





2) doc169#wc_1 1999-05-01 00:00:00





3) doc190#wc_1 1999-06-12 00:00:00





4) doc215#wc_1 1999-08-07 00:00:00





5) doc223#wc_1 1999-09-13 00:00:00


#### Print thread from keyword

In [45]:
keyword_df.head()

,threads,mean_weight
word,,
happy,1,-14.816993
james,1,-14.829668
called,1,-14.869827
fenwick,1,-14.869827
online,1,-14.883680


In [47]:
#Find Thread index associated with a keyword
keyword2thread["james"]

[225]

In [48]:
thread_idx=225
print_thread(threads,thread_idx,paragraphDF,data_5w1h,outside_text=True)

Thread length: 4
Period: 1995-06-17 00:00:00 to 1995-09-11 00:00:00

1) doc295#wc_2 1995-06-17 00:00:00
3. Charles suppose be caring if Larry could mail a copy past presentations to James, Keith, perhaps who Dick Chambers during later evaluation. And James, then pass on future remarks to Keith regarding how really sewing meetings up..





2) doc302#wc_2 1995-07-07 00:00:00
3. I only laugh relieved if Dad showed distribute round look of Harry to James, Mike, Steven und I<unk> for its family. John Philip, your sit upon my laugh at Andy Williams and still lighting my up..





3) doc322#wc_2 1995-07-19 00:00:00
3. Let Just be wondering cannot post even be a submission TO Parliament without James, PM, even noting his full or valid views. Again I, our all hundreds of regards acknowledge Keith for also sufficiently communicating her policy





4) doc371#wc_0 1995-09-11 00:00:00
1. This Board under reverse m) a proposed base capital list policy continue locked place in outlined last July ON GRAND FIRE DISTRICT PRO. On institutional model consistent as Number 8 signed agreed replies to Tom Patel, P(TT), documents provided self. In the correctly expect noted below shares are segregated. But review as the delegates explore common disclosure summary available board special basis affecting existing member shareholders replacing standard enclosed slides.


## Final Results

In [52]:
result_file=os.path.join(config.target_dir,"results.txt")

passage_count=len(read_json_dump(config.passage_dict_file))
passage_parts_count=len(read_parts(config.passage_parts_file))

with open(result_file,"w") as f:
    f.write(f"Total Passages={passage_count}\n")
    f.write(f"Total Passage Splits={passage_parts_count}\n\n")
    
    for res_file in glob.glob(os.path.join(config.threads_dir,'eval*.txt')):
        f.write("\n\n"+"="*5+res_file.split("/")[-1]+"="*5+"\n")
        with open(res_file,"r") as r:
            f.write(r.read()+"\n")

print(f"Result File: {result_file}\n")
with open(result_file,"r") as f:
    print(f.read())

Result File: /nfs/jup/sensitivity_classifier/threading/e2e_data/results.txt

Total Passages=1203
Total Passage Splits=4



=====eval_dpp.txt=====


Mean Cosine Similarity (MPDCS): 0.2939
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 112, Thread Sens Docs: 45
Thread Count: 40
Mean Thread Len: 2.8000
Mean Thread days span: 318.0250
All Sens Threads: 5
Thread Sens Ratios
	 0.00 - 0.25	: Count=13	Mean=0.0615	Std=0.0923
	 0.25 - 0.50	: Count=2	Mean=0.2917	Std=0.0417
	 0.50 - 0.75	: Count=20	Mean=0.5267	Std=0.0554
	 0.75 - 1.00	: Count=5	Mean=1.0000	Std=0.0000


=====eval_seqint_tfidflsa_w0.7_ward.txt=====


Mean Cosine Similarity (MPDCS): 0.3438
Total Docs: 1203, Total Sens Docs: 590
Thread Docs: 444, Thread Sens Docs: 213
Thread Count: 76
Mean Thread Len: 5.8421
Mean Thread days span: 1219.1974
All Sens Threads: 1
Thread Sens Ratios
	 0.00 - 0.25	: Count=9	Mean=0.1000	Std=0.0903
	 0.25 - 0.50	: Count=24	Mean=0.3585	Std=0.0497
	 0.50 - 0.75	: Count=34	Mean=0.5715	Std=0.0704
	 0.75 - 1